In [1]:
from llmcoder import LLMCoder

In [2]:
llmcoder = LLMCoder(analyzers=["mypy_analyzer_v1", "signature_analyzer_v1"], max_iter=3, feedback_variant="coworker")

In [3]:
code = ['''from llmcoder import LLMCoder

# Create an LLMCoder with a random seed of 0
llmcoder = ''',
'''import os
from aleph_alpha_client import Client, CompletionRequest, Prompt



client = Client(os.''', # https://github.com/Aleph-Alpha/aleph-alpha-client
'''# Import random forest regressor
''',
'''import os
import re
import subprocess
import tempfile

from llmcoder.analyze.Analyzer import Analyzer


class MypyAnalyzer(Analyzer):
    def analyze(self, input: str, completion: str, install_stubs: bool = True, *mypy_args: str | None) -> dict:
        code = input + completion

        with tempfile.NamedTemporaryFile(delete=False, suffix=".py", mode="w") as temp_file:
            temp_file_name = temp_file.name
            temp_file.write(code)

        if mypy_args is None:
            # mypy_args = ["--disable-error-code=import-untyped"]  #FiXME: Should resolve error: Skipping analyzing "sklearn.ensemble": module is installed, but missing library stubs or py.typed marker  [import-untyped] but doesn't
            mypy_args = []

        # First run of mypy to check for missing stubs
        mypy_run = subprocess.run(["mypy", temp_file_name, *mypy_args], capture_output=True, text=True)

        # Check if mypy reported missing stubs
        indicators = ["install-types"]
        missing_stubs = any(indicator in mypy_run.stdout or indicator in mypy_run.stderr for indicator in indicators)

        # Install stubs if missing
        if install_stubs and missing_stubs:
            print("Installing missing stubs...")

            # Install missing stubs
            subprocess.run(["mypy", "--install-types", "--non-interactive", *mypy_args], capture_output=True, text=True)

            # Re-run mypy after installing stubs
            mypy_run = subprocess.run(["mypy", temp_file_name, *mypy_args], capture_output=True, text=True)
        else:
            print("No missing stubs found.")

        result = mypy_run.stdout if mypy_run.stdout else mypy_run.stderr

        # Get the number of lines of the input code
        n_input_lines = len(input.split("\n")) - 1

        # Parse the line number from the mypy error message and filter the result
        filtered_result = []
        for line in result.split("\n"):
            if line.startswith(temp_file_name):
                line_number = int(line.split(":")[1])
                if line_number > n_input_lines:
                    filtered_result.append(line)
            else:
                filtered_result.append(line)

        # Replace the temp file name with "your completion". This helps the LLM understand that the error is caused by its completion.
        filtered_result = [line.replace(temp_file_name, "your completion") for line in filtered_result]

        # Remove the error message "your completion:2: \x1b[1m\x1b[31merror:\x1b[m Skipping analyzing..." since it cannot be resolved by installing stubs
        filter''',
'''import argparse

import uvicorn
from fastapi import APIRouter, FastAPI
from fastapi.middleware.cors import CORSMiddleware

from .routers import models, predict


def api(args: argparse.Namespace) -> FastAPI:
    app = FastAPI(
        title="PubMed Research Trends",
        description="PubMed Research Trends is a web application that allows users to explore trends in biomedical research. Users can search for topics and visualize the trends in the number of publications over time. The application also provides a topic clustering feature that allows users to explore the topics that are related to a given topic.",
        version="0.1.0",
    )

    app.add_middleware(''',
'''from langchain.document_loaders import DirectoryLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embed and store
from langchain.vectorstores.elasticsearch import ElasticsearchStore 
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OllamaEmbeddings # We can also try Ollama embeddings

from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

import os
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import time

import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# pip install jq
schema = {
    'jq_schema': '.text'
}

es = get_es_connection()

REINDEX = False

# pip install gpt4all
# Needs Ubuntu > 22.04 because of glibc

if REINDEX:
    # Clear the index
    es.indices.delete(index="eurlex-langchain", ignore=[400, 404])

    start_time = time.time()
    vectorstore = '''
]

In [4]:
result = llmcoder.complete(code[-1])

Creating first completion...
Starting feedback loop...
Starting feedback iteration 1...
Analyzing code in a coworker mode...
Running mypy_analyzer_v1...
No missing stubs found.
/tmp/tmpybr6z05x.py:27: error: Name "get_es_connection" is not defined  [name-defined]
/tmp/tmpybr6z05x.py:39: error: Too many positional arguments for "ElasticsearchStore"  [misc]
/tmp/tmpybr6z05x.py:39: error: Argument 2 to "ElasticsearchStore" has incompatible type "str"; expected "Embeddings | None"  [arg-type]
Found 3 errors in 1 file (checked 1 source file)

Running signature_analyzer_v1...
Using context from previous analyzers: ['mypy_analyzer_v1']
Found problematic function or class: ElasticsearchStore
Found problematic function or class: ElasticsearchStore
Name ElasticsearchStore matches query ['ElasticsearchStore']
Got signatures:
(index_name: str, *, embedding: Optional[langchain_core.embeddings.Embeddings] = None, es_connection: Optional[ForwardRef('Elasticsearch')] = None, es_url: Optional[str] = No

In [5]:
result

'ElasticsearchStore("eurlex-langchain", embedding=OllamaEmbeddings(), es_connection=es)\n    llm = Ollama(OllamaEmbeddings(), vectorstore)\n    llm.load("eurlex-ollama.v1", "/mnt/data/eurlex-ollama.v1")\n    print("Time to load LLM:", time.time()-start_time)'

In [6]:
import re

In [7]:
for message in llmcoder.messages:
    print(message['role'].upper())
    # Remove colors from the content of the message
    message['content'] = re.sub(r'\x1b[^m]*m', '', message['content'])
    print(message['content'])
    print('-' * 80)

SYSTEM
You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.
You are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.
Precisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.
Your code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.
Your unique capability is to provide completions without altering, repeating, or commenting on the original code.
You offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.
This approach makes you an ideal assistant for users seeking straightforward and efficient code extensions, enhancing their work with accurate, logic-driven compl

In [8]:
# Copilot:
# from llmcoder import LLMCoder

# # Create an LLMCoder with a random seed of 0
# llmcoder = LLMCoder(0)